In [1]:
import pandas as pd
import json

In [2]:
df_inference = pd.read_csv("gs://ds-training-ay3/lending_club/data/test_set.csv")

In [3]:
df_inference.head()

,Id,emp_length,annual_inc,zip_code,debt_to_income,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,...,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,policy_code_PC1,policy_code_PC2,policy_code_PC3,policy_code_PC4,policy_code_PC5
0,42,4,250000.0,88,0.98,0,2,120.0,0.0,6,...,0,0,0,0,0,0,0,1,0,0
1,71,1,48000.0,100,2.60,0,0,120.0,0.0,4,...,0,0,0,0,0,0,0,1,0,0
2,91,1,51400.0,223,1.77,0,2,120.0,0.0,7,...,0,0,0,0,0,0,0,1,0,0
3,102,3,61000.0,752,0.98,0,4,120.0,0.0,8,...,0,0,0,0,0,0,1,0,0,0
4,104,2,27600.0,310,21.83,0,0,120.0,0.0,5,...,0,0,0,0,0,1,0,0,0,0


In [4]:
inference_sample = df_inference.iloc[:10,:].reset_index(drop=True)
inference_sample.set_index(["Id"], inplace=True)

In [5]:
inference_sample.to_csv("gs://ds-training-ay3/lending_club/inference/inference_data.csv")

In [6]:
inference_sample.head()

,emp_length,annual_inc,zip_code,debt_to_income,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,...,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,policy_code_PC1,policy_code_PC2,policy_code_PC3,policy_code_PC4,policy_code_PC5
Id,,,,,,,,,,,,,,,,,,,,,
42,4,250000.0,88,0.98,0,2,120.0,0.0,6,0,...,0,0,0,0,0,0,0,1,0,0
71,1,48000.0,100,2.60,0,0,120.0,0.0,4,0,...,0,0,0,0,0,0,0,1,0,0
91,1,51400.0,223,1.77,0,2,120.0,0.0,7,0,...,0,0,0,0,0,0,0,1,0,0
102,3,61000.0,752,0.98,0,4,120.0,0.0,8,0,...,0,0,0,0,0,0,1,0,0,0
104,2,27600.0,310,21.83,0,0,120.0,0.0,5,0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
# parameters
LOCATION = "us-central1"
PROJECT = "ds-training-380514"
BATCH_JOB_NAME = "lending_batch"
MODEL_ID = "projects/354621994428/locations/us-central1/models/8670847652688035840"
URI = "gs://ds-training-ay3/lending_club/inference/inference_data.csv"
OUTPUT_URI_PREFIX = "gs://ds-training-ay3/lending_club/batch_predict_output"
MACHINE_TYPE = "e2-standard-2"
STARTING_REPLICA_COUNT = 1
MAX_REPLICA_COUNT = 10  # default
GENERATE_EXPLANATION = "false"  # default value

In [8]:
request_body = {
    "displayName": BATCH_JOB_NAME,
    "model": MODEL_ID,
    "inputConfig": {
        "instancesFormat": "csv",
        "gcsSource": {
            "uris": [
                URI
            ]
        },
    },
    "outputConfig": {
        "predictionsFormat": "jsonl",
        "gcsDestination": {
            "outputUriPrefix": OUTPUT_URI_PREFIX
        }
    },
    "dedicatedResources": {
        "machineSpec": {
            "machineType": MACHINE_TYPE,
            "acceleratorCount": "0"
        },
        "startingReplicaCount": STARTING_REPLICA_COUNT,
        "maxReplicaCount": MAX_REPLICA_COUNT
    },
    "generateExplanation": GENERATE_EXPLANATION
}

In [9]:
request_body

{'displayName': 'lending_batch',
 'model': 'projects/354621994428/locations/us-central1/models/8670847652688035840',
 'inputConfig': {'instancesFormat': 'csv',
  'gcsSource': {'uris': ['gs://ds-training-ay3/lending_club/inference/inference_data.csv']}},
 'outputConfig': {'predictionsFormat': 'jsonl',
  'gcsDestination': {'outputUriPrefix': 'gs://ds-training-ay3/lending_club/batch_predict_output'}},
 'dedicatedResources': {'machineSpec': {'machineType': 'e2-standard-2',
   'acceleratorCount': '0'},
  'startingReplicaCount': 1,
  'maxReplicaCount': 10},
 'generateExplanation': 'false'}

In [10]:
with open("request.json", "w") as outfile:
    outfile.write(json.dumps(request_body))

In [15]:
region='us-central1'
project = 'ds-training-380514'

In [16]:
api_endpoint = f"https://{region}-aiplatform.googleapis.com/v1/projects/{project}/locations/{region}/batchPredictionJobs"

In [17]:
!curl -X POST \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "Content-Type: application/json; charset=utf-8" \
    -d @request.json \
    {api_endpoint}

{
  "name": "projects/354621994428/locations/us-central1/batchPredictionJobs/7000868023296851968",
  "displayName": "lending_batch",
  "model": "projects/354621994428/locations/us-central1/models/8670847652688035840",
  "inputConfig": {
    "instancesFormat": "csv",
    "gcsSource": {
      "uris": [
        "gs://ds-training-ay3/lending_club/inference/inference_data.csv"
      ]
    }
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://ds-training-ay3/lending_club/batch_predict_output"
    }
  },
  "dedicatedResources": {
    "machineSpec": {
      "machineType": "e2-standard-2"
    },
    "startingReplicaCount": 1,
    "maxReplicaCount": 10
  },
  "manualBatchTuningParameters": {},
  "state": "JOB_STATE_PENDING",
  "createTime": "2023-04-05T06:39:43.391425Z",
  "updateTime": "2023-04-05T06:39:43.391425Z",
  "modelVersionId": "1"
}


In [18]:
!gsutil cp gs://ds-training-ay3/lending_club/batch_prediction_output/prediction-ml_engine_us-central1_census_income_student_1_v1-2023_04_04T17_41_37_528Z/prediction.results-00000-of-00002 .

Copying gs://ds-training-ay3/lending_club/batch_prediction_output/prediction-ml_engine_us-central1_census_income_student_1_v1-2023_04_04T17_41_37_528Z/prediction.results-00000-of-00002...
/ [1 files][  2.4 KiB/  2.4 KiB]                                                
Operation completed over 1 objects/2.4 KiB.                                      


In [19]:
import pandas as pd 
file_path = "gs://ds-training-ay3/lending_club/batch_prediction_output/prediction-ml_engine_us-central1_census_income_student_1_v1-2023_04_04T17_41_37_528Z/prediction.results-00000-of-00002"
file_path = "prediction.results-00000-of-00002"
jsonObj = pd.read_json(path_or_buf=file_path, lines=True)
jsonObj

,instance,prediction
0,"[4.0, 104.0, 2.0, 27600.0, 310.0, 21.83, 0.0, ...",[]
1,"[1.0, 71.0, 1.0, 48000.0, 100.0, 2.6, 0.0, 0.0...",[]
2,"[5.0, 133.0, 1.0, 65000.0, 606.0, 3.01, 0.0, 1...",[]
3,"[0.0, 42.0, 4.0, 250000.0, 88.0, 0.98, 0.0, 2....",[]
4,"[7.0, 269.0, 3.0, 48000.0, 980.0, 6.25, 1.0, 0...",[]
